# Sudoku-6 : Resolution par CSP Academique (Python)

**Niveau** : Programmation par Contraintes | **Duree** : ~25 min | **Prerequis** : Sudoku-0 Environment

## Navigation

| << Precedent | [Index](README.md) | Suivant >> |
|-------------|---------------------|-----------|
| [Sudoku-5-PSO-Python](Sudoku-5-PSO-Python.ipynb) | | [Sudoku-8-HumanStrategies-Python](Sudoku-8-HumanStrategies-Python.ipynb) |

## Objectifs d'apprentissage

A la fin de ce notebook, vous saurez :
1. **Formaliser** le Sudoku comme un CSP (variables, domaines, contraintes)
2. **Implementer** les algorithmes de reference AIMA : AC-3, Forward Checking, MAC
3. **Appliquer** les heuristiques MRV et LCV pour optimiser la recherche
4. **Comparer** experimentalement les differentes strategies de resolution

---

Ce notebook presente la resolution de Sudoku selon l'approche academique decrite dans **"Artificial Intelligence: A Modern Approach"** (Russell & Norvig, Chapitre 6).

## Introduction

Contrairement aux bibliothèques industrielles (OR-Tools, Choco) ou aux metaheuristiques (GA, SA, PSO), l'approche AIMA :
- **Est pedagogique** : chaque composant est transparent et comprehensible
- **Est modulaire** : on peut combiner differentes heuristiques et propagations
- **Sert de reference** : c'est le standard academique pour comparer les algorithmes

### Formalisation CSP du Sudoku

| Composant | Description | Taille |
|-----------|-------------|-------|
| **Variables** | $X_{i,j}$ pour chaque cellule (i, j) | 81 |
| **Domaines** | $D_{i,j} \subseteq \{1, 2, \ldots, 9\}$ | 9 valeurs max |
| **Contraintes** | AllDifferent par ligne, colonne, bloc 3x3 | 27 contraintes |

### Algorithmes couverts

| Algorithme | Type | Complexite | Puissance |
|------------|------|------------|----------|
| **Backtracking simple** | Recherche | $O(d^n)$ | Faible |
| **Backtracking + MRV** | Heuristique | Variable | Moderee |
| **Forward Checking** | Propagation 1-niveau | $O(n \cdot d^2)$ | Moderee |
| **AC-3** | Arc-consistance | $O(e \cdot d^3)$ | Forte |
| **MAC** | AC-3 + Backtracking | Variable | Tres forte |

In [ ]:
# Imports
import numpy as np
import time
import copy
from typing import List, Tuple, Optional, Dict, Set, Callable, Any
from collections import defaultdict
from enum import Enum

print("Libraries importees avec succes.")

In [ ]:
# Configuration du chemin vers les puzzles
import os
from pathlib import Path

NOTEBOOK_DIR = Path(r"D:\Dev\CoursIA\MyIA.AI.Notebooks\Sudoku")
PUZZLES_DIR = NOTEBOOK_DIR / "Puzzles"

if PUZZLES_DIR.exists():
    print(f"Dossier Puzzles: {PUZZLES_DIR}")
else:
    print(f"ATTENTION: Dossier Puzzles non trouve a {PUZZLES_DIR}")
    PUZZLES_DIR = Path(os.getcwd()) / "Puzzles"

## 1. Classe SudokuGrid

In [ ]:
class SudokuGrid:
    """Representation d'une grille de Sudoku 9x9."""
    
    def __init__(self, grid: Optional[List[List[int]]] = None):
        if grid is None:
            self.cells = [[0] * 9 for _ in range(9)]
        else:
            self.cells = [row[:] for row in grid]
    
    @classmethod
    def from_string(cls, s: str) -> 'SudokuGrid':
        s = s.replace('.', '0').replace(' ', '').replace('\n', '')
        if len(s) != 81:
            raise ValueError(f"La chaine doit avoir 81 caracteres")
        grid = cls()
        for i in range(81):
            grid.cells[i // 9][i % 9] = int(s[i])
        return grid
    
    def clone(self) -> 'SudokuGrid':
        return SudokuGrid(self.cells)
    
    def to_string(self) -> str:
        return ''.join(str(self.cells[r][c]) for r in range(9) for c in range(9))
    
    def is_valid(self) -> bool:
        """Verifie si la grille est valide (sans doublons)."""
        for i in range(9):
            # Lignes
            row = [v for v in self.cells[i] if v != 0]
            if len(row) != len(set(row)):
                return False
            # Colonnes
            col = [self.cells[r][i] for r in range(9) if self.cells[r][i] != 0]
            if len(col) != len(set(col)):
                return False
        # Blocs
        for br in range(3):
            for bc in range(3):
                block = []
                for r in range(br*3, br*3+3):
                    for c in range(bc*3, bc*3+3):
                        if self.cells[r][c] != 0:
                            block.append(self.cells[r][c])
                if len(block) != len(set(block)):
                    return False
        return True
    
    def is_complete(self) -> bool:
        """Verifie si la grille est complete (pas de 0)."""
        return all(self.cells[r][c] != 0 for r in range(9) for c in range(9))
    
    def __str__(self) -> str:
        lines = []
        for r in range(9):
            if r > 0 and r % 3 == 0:
                lines.append('-' * 21)
            row_str = ''
            for c in range(9):
                if c > 0 and c % 3 == 0:
                    row_str += '| '
                val = self.cells[r][c]
                row_str += (str(val) if val != 0 else '.') + ' '
            lines.append(row_str)
        return '\n'.join(lines)

def load_puzzles(filepath: str, max_puzzles: int = None) -> List[str]:
    puzzles = []
    with open(filepath, 'r') as f:
        for line in f:
            line = line.strip()
            if len(line) >= 81:
                puzzles.append(line[:81])
                if max_puzzles and len(puzzles) >= max_puzzles:
                    break
    return puzzles

# Charger les puzzles
easy_puzzles = load_puzzles(str(PUZZLES_DIR / 'Sudoku_Easy51.txt'), max_puzzles=5)
print(f"Puzzles charges: {len(easy_puzzles)}")

test_grid = SudokuGrid.from_string(easy_puzzles[0])
print("\nGrille de test:")
print(test_grid)

## 2. Classe CSP Generique

Nous definissons une classe CSP generique inspiree du livre AIMA. Cette classe represente un **CSP binaire** (contraintes entre paires de variables).

In [ ]:
class CSP:
    """Probleme de Satisfaction de Contraintes (CSP) binaire.
    
    Inspire de AIMA - Russell & Norvig, Chapitre 6.
    """
    
    def __init__(self, variables: List[Any], domains: Dict[Any, List[Any]],
                 neighbors: Dict[Any, List[Any]], 
                 constraint_func: Callable[[Any, Any, Any, Any], bool]):
        self.variables = variables
        self.domains = {v: list(d) for v, d in domains.items()}
        self.neighbors = {v: list(n) for v, n in neighbors.items()}
        self.constraint_func = constraint_func
        
        # Compteurs pour l'analyse
        self.num_assignments = 0
        self.num_backtracks = 0
    
    def is_consistent(self, var: Any, val: Any, assignment: Dict[Any, Any]) -> bool:
        """Verifie si (var, val) est consistant avec l'assignation partielle."""
        for neighbor in self.neighbors[var]:
            if neighbor in assignment:
                if not self.constraint_func(var, val, neighbor, assignment[neighbor]):
                    return False
        return True
    
    def is_complete(self, assignment: Dict[Any, Any]) -> bool:
        """Verifie si l'assignation est complete."""
        return len(assignment) == len(self.variables)
    
    def copy_domains(self) -> Dict[Any, List[Any]]:
        """Retourne une copie profonde des domaines."""
        return {v: list(d) for v, d in self.domains.items()}
    
    def get_arcs(self) -> List[Tuple[Any, Any]]:
        """Retourne tous les arcs (Xi, Xj) du CSP."""
        arcs = []
        for var in self.variables:
            for neighbor in self.neighbors[var]:
                arcs.append((var, neighbor))
        return arcs

print("Classe CSP definie.")

## 3. Construction du CSP Sudoku

Nous transformons une grille Sudoku en instance CSP avec :
- **81 variables** : une par cellule (0,0) a (8,8)
- **Domaines** : {1..9} pour les cellules vides, {v} pour les cellules fixees
- **Contraintes** : AllDifferent representee comme paires binaires !=

In [ ]:
class SudokuCSPBuilder:
    """Constructeur de CSP a partir d'une grille Sudoku."""
    
    @staticmethod
    def build_csp(grid: SudokuGrid) -> CSP:
        # Variables : (row, col) pour chaque cellule
        variables = [(i, j) for i in range(9) for j in range(9)]
        
        # Domaines : 1-9 pour les vides, valeur unique pour les fixees
        domains = {}
        for i in range(9):
            for j in range(9):
                value = grid.cells[i][j]
                if value == 0:
                    domains[(i, j)] = list(range(1, 10))
                else:
                    domains[(i, j)] = [value]
        
        # Voisins : meme ligne, meme colonne, meme bloc
        neighbors = {}
        for i in range(9):
            for j in range(9):
                neighbor_set = set()
                
                # Meme ligne
                for k in range(9):
                    if k != j:
                        neighbor_set.add((i, k))
                
                # Meme colonne
                for k in range(9):
                    if k != i:
                        neighbor_set.add((k, j))
                
                # Meme bloc 3x3
                block_row = (i // 3) * 3
                block_col = (j // 3) * 3
                for r in range(block_row, block_row + 3):
                    for c in range(block_col, block_col + 3):
                        if r != i or c != j:
                            neighbor_set.add((r, c))
                
                neighbors[(i, j)] = list(neighbor_set)
        
        # Fonction de contrainte : valeurs differentes
        def constraint(v1: Tuple[int, int], val1: int, 
                     v2: Tuple[int, int], val2: int) -> bool:
            return val1 != val2
        
        return CSP(variables, domains, neighbors, constraint)
    
    @staticmethod
    def apply_solution(grid: SudokuGrid, assignment: Dict[Tuple[int, int], int]) -> None:
        """Apique une solution CSP a une grille Sudoku."""
        for (i, j), value in assignment.items():
            grid.cells[i][j] = value

print("SudokuCSPBuilder defini.")

## 4. Backtracking Simple

In [ ]:
class BacktrackingSimple:
    """Backtracking simple pour CSP."""
    
    @staticmethod
    def solve(csp: CSP, assignment: Optional[Dict] = None) -> Optional[Dict]:
        if assignment is None:
            assignment = {}
        
        if csp.is_complete(assignment):
            return assignment
        
        # Choisir la premiere variable non assignee (ordre naif)
        unassigned = [v for v in csp.variables if v not in assignment]
        var = unassigned[0]
        
        for val in csp.domains[var]:
            csp.num_assignments += 1
            
            if csp.is_consistent(var, val, assignment):
                assignment[var] = val
                result = BacktrackingSimple.solve(csp, assignment)
                if result is not None:
                    return result
                assignment.pop(var)
                csp.num_backtracks += 1
        
        return None

print("BacktrackingSimple defini.")

## 5. Heuristiques : MRV et LCV

### MRV (Minimum Remaining Values)
Heuristique de **selection de variable** : choisir la variable avec le plus petit domaine restant.

### LCV (Least Constraining Value)
Heuristique d'**ordonnancement des valeurs** : essayer d'abord la valeur qui elimine le moins de possibilites chez les voisins.

In [ ]:
class CSPHeuristics:
    """Heuristiques pour la resolution CSP."""
    
    @staticmethod
    def select_mrv(csp: CSP, assignment: Dict, 
                    current_domains: Optional[Dict] = None) -> Any:
        """MRV : Selectionne la variable avec le moins de valeurs viables.
        
        En cas d'egalite, utilise le degre (nombre de voisins non assignes).
        """
        if current_domains is None:
            current_domains = csp.domains
        
        unassigned = [v for v in csp.variables if v not in assignment]
        
        def remaining_values(v):
            return sum(1 for val in current_domains[v] 
                      if csp.is_consistent(v, val, assignment))
        
        def degree(v):
            return sum(1 for n in csp.neighbors[v] if n not in assignment)
        
        # MRV croissant, puis degre decroissant
        return min(unassigned, key=lambda v: (remaining_values(v), -degree(v)))
    
    @staticmethod
    def order_lcv(csp: CSP, var: Any, assignment: Dict,
                   current_domains: Optional[Dict] = None) -> List[Any]:
        """LCV : Ordonne les valeurs par nombre de conflits croissant."""
        if current_domains is None:
            current_domains = csp.domains
        
        def conflicts(val):
            count = 0
            for neighbor in csp.neighbors[var]:
                if neighbor not in assignment:
                    for nval in current_domains[neighbor]:
                        if not csp.constraint_func(var, val, neighbor, nval):
                            count += 1
            return count
        
        return sorted(current_domains[var], key=conflicts)

print("CSPHeuristics defini.")

## 6. Backtracking Ameliore (MRV + LCV)

In [ ]:
class BacktrackingImproved:
    """Backtracking avec heuristiques MRV et LCV."""
    
    @staticmethod
    def solve(csp: CSP, assignment: Optional[Dict] = None,
               current_domains: Optional[Dict] = None,
               use_mrv: bool = True, use_lcv: bool = True) -> Optional[Dict]:
        if assignment is None:
            assignment = {}
        if current_domains is None:
            current_domains = csp.copy_domains()
        
        if csp.is_complete(assignment):
            return assignment
        
        # Selection de variable
        var = (CSPHeuristics.select_mrv(csp, assignment, current_domains) 
               if use_mrv else [v for v in csp.variables if v not in assignment][0])
        
        # Ordonnancement des valeurs
        values = (CSPHeuristics.order_lcv(csp, var, assignment, current_domains)
                  if use_lcv else current_domains[var])
        
        for val in values:
            csp.num_assignments += 1
            
            if csp.is_consistent(var, val, assignment):
                assignment[var] = val
                result = BacktrackingImproved.solve(csp, assignment, current_domains, 
                                                       use_mrv, use_lcv)
                if result is not None:
                    return result
                assignment.pop(var)
                csp.num_backtracks += 1
        
        return None

print("BacktrackingImproved defini.")

## 7. Forward Checking

Le **Forward Checking** propage l'assignation d'une variable vers ses voisins immediats, reduisant leurs domaines et detectant les echecs plus tot.

In [ ]:
class ForwardChecking:
    """Forward Checking : propage l'assignation vers les voisins."""
    
    @staticmethod
    def propagate(csp: CSP, var: Any, val: Any, assignment: Dict,
                  current_domains: Dict) -> Tuple[List[Tuple[Any, Any]], bool]:
        """Propage l'assignation var=val vers les voisins non assignes.
        
        Returns:
            (removals, success): Liste des valeurs retirees et succes
        """
        removals = []
        
        for neighbor in csp.neighbors[var]:
            if neighbor not in assignment:
                to_remove = []
                for nval in current_domains[neighbor]:
                    if not csp.constraint_func(var, val, neighbor, nval):
                        to_remove.append(nval)
                        removals.append((neighbor, nval))
                
                for r in to_remove:
                    current_domains[neighbor].remove(r)
                
                if len(current_domains[neighbor]) == 0:
                    return removals, False  # Domaine vide = echec
        
        return removals, True
    
    @staticmethod
    def restore(current_domains: Dict, removals: List[Tuple[Any, Any]]) -> None:
        """Restaure les valeurs retirees."""
        for var, val in removals:
            current_domains[var].append(val)
    
    @staticmethod
    def solve(csp: CSP, assignment: Optional[Dict] = None,
               current_domains: Optional[Dict] = None) -> Optional[Dict]:
        if assignment is None:
            assignment = {}
        if current_domains is None:
            current_domains = csp.copy_domains()
        
        if csp.is_complete(assignment):
            return assignment
        
        var = CSPHeuristics.select_mrv(csp, assignment, current_domains)
        
        for val in CSPHeuristics.order_lcv(csp, var, assignment, current_domains):
            csp.num_assignments += 1
            
            if csp.is_consistent(var, val, assignment):
                assignment[var] = val
                
                removals, success = ForwardChecking.propagate(
                    csp, var, val, assignment, current_domains
                )
                
                if success:
                    result = ForwardChecking.solve(csp, assignment, current_domains)
                    if result is not None:
                        return result
                
                ForwardChecking.restore(current_domains, removals)
                assignment.pop(var)
                csp.num_backtracks += 1
        
        return None

print("ForwardChecking defini.")

## 8. Arc Consistency (AC-3)

L'algorithme **AC-3** assure que pour chaque arc (Xi, Xj), toute valeur de Xi a un support dans Xj.

In [ ]:
class AC3:
    """Algorithme AC-3 pour la consistance d'arc."""
    
    @staticmethod
    def revise(csp: CSP, xi: Any, xj: Any, 
               current_domains: Dict) -> bool:
        """Rend l'arc (xi, xj) arc-consistent.
        
        Returns:
            True si le domaine de xi a ete modifie
        """
        revised = False
        to_remove = []
        
        for val_i in current_domains[xi]:
            # Chercher un support dans xj
            has_support = any(
                csp.constraint_func(xi, val_i, xj, val_j)
                for val_j in current_domains[xj]
            )
            
            if not has_support:
                to_remove.append(val_i)
                revised = True
        
        for val in to_remove:
            current_domains[xi].remove(val)
        
        return revised
    
    @staticmethod
    def run(csp: CSP, current_domains: Dict,
             arcs: Optional[List[Tuple[Any, Any]]] = None) -> bool:
        """Rend le CSP arc-consistent.
        
        Returns:
            False si un domaine devient vide (echec)
        """
        from collections import deque
        
        queue = deque(arcs if arcs else csp.get_arcs())
        
        while queue:
            xi, xj = queue.popleft()
            
            if AC3.revise(csp, xi, xj, current_domains):
                if len(current_domains[xi]) == 0:
                    return False  # Domaine vide = echec
                
                # Ajouter les arcs (xk, xi) pour k != j
                for xk in csp.neighbors[xi]:
                    if xk != xj:
                        queue.append((xk, xi))
        
        return True

print("AC3 defini.")

## 9. MAC (Maintaining Arc Consistency)

L'algorithme **MAC** combine le backtracking avec AC-3 : apres chaque assignation, on maintient la consistance d'arc sur tout le CSP.

In [ ]:
class MAC:
    """MAC (Maintaining Arc Consistency) : Backtracking + AC-3."""
    
    @staticmethod
    def solve(csp: CSP, assignment: Optional[Dict] = None,
               current_domains: Optional[Dict] = None) -> Optional[Dict]:
        if assignment is None:
            assignment = {}
        if current_domains is None:
            current_domains = csp.copy_domains()
        
        if csp.is_complete(assignment):
            return assignment
        
        var = CSPHeuristics.select_mrv(csp, assignment, current_domains)
        
        for val in CSPHeuristics.order_lcv(csp, var, assignment, current_domains):
            csp.num_assignments += 1
            
            if csp.is_consistent(var, val, assignment):
                assignment[var] = val
                
                # Sauvegarder les domaines
                saved_domains = {v: list(d) for v, d in current_domains.items()}
                
                # Reduire le domaine a {val}
                current_domains[var] = [val]
                
                # Executer AC-3 sur les arcs affectes
                arcs = [(n, var) for n in csp.neighbors[var] if n not in assignment]
                success = AC3.run(csp, current_domains, arcs)
                
                if success:
                    result = MAC.solve(csp, assignment, current_domains)
                    if result is not None:
                        return result
                
                # Restaurer les domaines
                for v, d in saved_domains.items():
                    current_domains[v] = d
                assignment.pop(var)
                csp.num_backtracks += 1
        
        return None

print("MAC defini.")

## 10. Test et Benchmark

In [ ]:
# Test sur un puzzle
puzzle = SudokuGrid.from_string(easy_puzzles[0])
print("Puzzle original:")
print(puzzle)

# Creer le CSP
csp = SudokuCSPBuilder.build_csp(puzzle)

# Tester avec MAC
start = time.time()
solution = MAC.solve(csp)
elapsed = time.time() - start

if solution:
    result = puzzle.clone()
    SudokuCSPBuilder.apply_solution(result, solution)
    print(f"\nSolution (MAC): {elapsed*1000:.1f}ms, {csp.num_assignments} assignations, {csp.num_backtracks} backtracks")
    print(result)
    print(f"\nSolution valide: {result.is_valid()}")
else:
    print("Pas de solution trouvee")

### Interpretation : Resultat MAC

La solution est trouvee presque instantanement. L'algorithme MAC :
1. Utilise MRV pour choisir la cellule la plus contrainte
2. Utilise LCV pour essayer les valeurs les moins contraignantes
3. Maintient la consistance d'arc apres chaque assignation

In [ ]:
# Comparaison des strategies
from enum import Enum

class CSPStrategy(Enum):
    BACKTRACKING_SIMPLE = 1
    BACKTRACKING_MRV_LCV = 2
    FORWARD_CHECKING = 3
    MAC = 4

def solve_with_strategy(grid: SudokuGrid, strategy: CSPStrategy) -> Tuple[Optional[SudokuGrid], int, int, float]:
    """Resout avec une strategie donnee."""
    csp = SudokuCSPBuilder.build_csp(grid)
    
    start = time.time()
    
    if strategy == CSPStrategy.BACKTRACKING_SIMPLE:
        solution = BacktrackingSimple.solve(csp)
    elif strategy == CSPStrategy.BACKTRACKING_MRV_LCV:
        solution = BacktrackingImproved.solve(csp, use_mrv=True, use_lcv=True)
    elif strategy == CSPStrategy.FORWARD_CHECKING:
        solution = ForwardChecking.solve(csp)
    elif strategy == CSPStrategy.MAC:
        solution = MAC.solve(csp)
    else:
        raise ValueError(f"Strategie inconnue: {strategy}")
    
    elapsed = time.time() - start
    
    if solution:
        result = grid.clone()
        SudokuCSPBuilder.apply_solution(result, solution)
        return result, csp.num_assignments, csp.num_backtracks, elapsed
    
    return None, csp.num_assignments, csp.num_backtracks, elapsed

# Benchmark
strategies = [
    CSPStrategy.BACKTRACKING_SIMPLE,
    CSPStrategy.BACKTRACKING_MRV_LCV,
    CSPStrategy.FORWARD_CHECKING,
    CSPStrategy.MAC
]

print("\n=== Benchmark sur 3 puzzles faciles ===")
print("=" * 80)
print(f"{'Strategie':<25} {'Assigns':>10} {'Backtracks':>12} {'Temps(ms)':>12} {'Succes':>8}")
print("-" * 80)

for strategy in strategies:
    total_assigns = 0
    total_backtracks = 0
    total_time = 0
    successes = 0
    
    for i in range(min(3, len(easy_puzzles))):
        grid = SudokuGrid.from_string(easy_puzzles[i])
        result, assigns, backtracks, elapsed = solve_with_strategy(grid, strategy)
        
        total_assigns += assigns
        total_backtracks += backtracks
        total_time += elapsed
        if result and result.is_valid():
            successes += 1
    
    name = strategy.name.replace('_', ' ')
    print(f"{name:<25} {total_assigns//3:>10} {total_backtracks//3:>12} {total_time*1000:>12.0f} {successes}/3")

print("=" * 80)

### Interpretation : Comparaison des strategies

| Strategie | Assignations | Backtracks | Analyse |
|-----------|-------------|------------|----------|
| **Simple** | Eleve | Eleve | Ne profite d'aucune optimisation |
| **MRV+LCV** | Reduit | Reduit | Fail-first + succeed-first |
| **Forward Checking** | Tres reduit | Tres reduit | Propagation 1-niveau |
| **MAC** | Minimal | Minimal | Propagation complete |

**Observations cles** :
1. **MRV** est l'heuristique la plus impactante (reduction souvent > 10x)
2. **FC** ajoute un gain significatif en detectant les echecs plus tot
3. **MAC** est optimal mais plus couteux par noeud (AC-3 a chaque pas)

## 11. Exercices

### Exercice 1 : Comptage des contraintes
Combien de contraintes binaires contient le CSP du Sudoku ? Verifiez votre reponse en comptant les arcs.

<details>
<summary><b>Solution</b></summary>

Chaque cellule a 20 voisins (8 meme ligne + 8 meme colonne + 4 meme bloc hors ligne/colonne).
Il y a 81 cellules, donc 81 x 20 = 1620 arcs orientes, soit 810 contraintes binaires.

```python
csp = SudokuCSPBuilder.build_csp(puzzle)
arcs = csp.get_arcs()
print(f"Nombre d'arcs: {len(arcs)}")  # 1620
print(f"Nombre de contraintes: {len(arcs) // 2}")  # 810
```
</details>

### Exercice 2 : AC-3 seul
AC-3 peut parfois resoudre un Sudoku completement sans backtracking. Testez sur un puzzle facile.

### Exercice 3 : Visualisation de la propagation
Modifiez le solveur MAC pour afficher la taille des domaines apres chaque assignation.

## Resume

### Algorithmes implementes

| Algorithme | Propagation | Detection d'echec | Performance |
|------------|-------------|-------------------|-------------|
| **Backtracking** | Aucune | A l'assignation | Lente |
| **BT + MRV + LCV** | Aucune | A l'assignation | Amelioree |
| **Forward Checking** | 1 niveau | Domaine voisin vide | Rapide |
| **MAC** | Complete (AC-3) | Domaine vide global | Optimale |

### Heuristiques

| Heuristique | Role | Effet |
|-------------|------|-------|
| **MRV** | Selection de variable | Fail-first |
| **LCV** | Ordonnancement valeurs | Succeed-first |
| **Degree** | Departage MRV | Priorite aux variables contraintes |

### Liens avec les autres notebooks

- **Sudoku-8-HumanStrategies** : Propagation plus poussees (naked/hidden singles)
- **Sudoku-10-ORTools** : Bibliotheque industrielle avec propagation optimisee
- **Sudoku-12-Z3** : SMT solver

### References

- Russell, S. & Norvig, P. *Artificial Intelligence: A Modern Approach*, 4e ed., Chapitre 6
- Mackworth, A. K. *Consistency in Networks of Relations* (1977)
- Dechter, R. *Constraint Processing*, Cambridge University Press, 2003

---

**Navigation** : [<< Sudoku-5-PSO](Sudoku-5-PSO-Python.ipynb) | [Index](README.md) | [Sudoku-8-HumanStrategies >>](Sudoku-8-HumanStrategies-Python.ipynb)